## Read in & concatenate data, clean 'datestop' column

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime as dt

Data: http://www1.nyc.gov/site/nypd/stats/reports-analysis/stopfrisk.page

Data dictionary: https://data.cityofnewyork.us/api/views/ftxv-d5ix/files/9cecce6d-baca-4a60-be9d-31501b5639f0

In [2]:
# NYPD released the data sets in three different formats: .zip, .csv, .xlsx
# create lists of parts to use in URL formation

base_link_zip = ['zip', '-csv.zip']

base_link_csv = ['excel', '.csv']

base_link_excel = ['excel', '.xlsx']

rename_dict = {
    #  in the middle of Bloomberg, one year some column names changed
    # 'forceuse' started in 2011
    'strname' : 'stname', 
    'rescod' : 'rescode', 
    'adrpct' : 'addrpct', 
    'prenam' : 'premname', 
    'strintr' : 'stinter', 
    'premtyp' : 'premtype', 
    'adrnum' : 'addrnum', 
    
    ## De Blasio change
    'STOP_FRISK_DATE' : 'datestop',
    'YEAR2' : 'year',
    'STOP_FRISK_TIME' : 'timestop',
    'RECORD_STATUS_CODE' : 'recstat',
    'LOCATION_IN_OUT_CODE' : 'inout',
    'OBSERVED_DURATION_MINUTES' : 'perobs',
    'SUSPECTED_CRIME_DESCRIPTION' : 'crimsusp',
    'STOP_DURATION_MINUTES' : 'perstop',
    'OFFICER_EXPLAINED_STOP_FLAG' : 'explnstp',
    'OTHER_PERSON_STOPPED_FLAG' : 'othpers',
    'SUSPECT_ARRESTED_FLAG' : 'arstmade',
    'SUSPECT_ARREST_OFFENSE' : 'arstoffn',
    'SUMMONS_ISSUED_FLAG' : 'sumissue',
    'SUMMONS_OFFENSE_DESCRIPTION' : 'sumoffen',
    'OFFICER_IN_UNIFORM_FLAG' : 'offunif',
    'SHIELD_IDENTIFIES_OFFICER_FLAG' : 'offshld',
    'VERBAL_IDENTIFIES_OFFICER_FLAG' : 'offverb',
    'ID_CARD_IDENTIFIES_OFFICER_FLAG' : 'officrid',
    'FRISKED_FLAG' : 'frisked',
    'SEARCHED_FLAG' : 'searched',
    'OTHER_CONTRABAND_FLAG' : 'contrabn',
    'SUSPECT_HEIGHT' : 'ht_feet', 
    'SUSPECT_WEIGHT' : 'weight',
    'SUSPECT_SEX' : 'sex',
    'KNIFE_CUTTER_FLAG' : 'knifcuti',
    'STOP_LOCATION_X' : 'xcoord',
    'STOP_LOCATION_Y' : 'ycoord',
    'SUSPECT_EYE_COLOR' : 'eyecolor',
    'SUSPECT_HAIR_COLOR' : 'haircolr',
    'SUSPECT_REPORTED_AGE' : 'age',
    'PHYSICAL_FORCE_HANDCUFF_SUSPECT_FLAG' : 'pf_hcuff',
    'SUSPECT_BODY_BUILD_TYPE' : 'build',
    'STOP_LOCATION_PRECINCT' : 'pct',
    'SEARCH_BASIS_ADMISSION_FLAG' : 'sb_admis',
    'SUSPECTS_ACTIONS_LOOKOUT_FLAG' : 'cs_lkout',
    'PHYSICAL_FORCE_OC_SPRAY_USED_FLAG' : 'pf_pepsp',
    'PHYSICAL_FORCE_DRAW_POINT_FIREARM_FLAG' : 'pf_drwep',
    'PHYSICAL_FORCE_WEAPON_IMPACT_FLAG' : 'pf_baton',
    'STOP_LOCATION_APARTMENT' : 'addrnum',
    'SUSPECT_RACE_DESCRIPTION' : 'race',
    'SEARCH_BASIS_HARD_OBJECT_FLAG' : 'sb_hdobj',
    'SUSPECTS_ACTIONS_CASING_FLAG' : 'cs_casng',
    'BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG' : 'cs_vcrim',
    'SEARCH_BASIS_OUTLINE_FLAG' : 'sb_outln',
    'SEARCH_BASIS_OTHER_FLAG' : 'sb_other',
    'STOP_LOCATION_BORO_NAME' : 'city',
    'SUSPECTS_ACTIONS_DRUG_TRANSACTIONS_FLAG' : 'cs_drgtr',
    'STOP_LOCATION_PREMISES_NAME' : 'premname',
    'STOP_LOCATION_STREET_NAME' : 'stname',
    'SUSPECTS_ACTIONS_DECRIPTION_FLAG' : 'cs_descr',
    'SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG' : 'cs_bulge',
    'JURISDICTION_CODE' : 'trhsloc'
}

drops = [
    'lineCM', 
    'detailCM', 
    'dettypCM',
    'wepfound',
    'detail1_',
    'linecm',
    'dettypcm', 
    'detailcm',
    'dettyp_c',
    'details_',
]

years = range(2003, 2020) 

all_columns = []

df = pd.DataFrame()
for year in years:
    firstpart = 'https://www1.nyc.gov/assets/nypd/downloads/'
    secondpart = '/analysis_and_planning/stop-question-frisk/sqf-'
    if year in range(2003, 2015):
        URL = ''.join([firstpart, base_link_zip[0], secondpart, str(year), base_link_zip[1]])
        lil_df = pd.read_csv(URL, encoding='ISO-8859-1')
    elif year in range(2015, 2017):
        URL = ''.join([firstpart, base_link_csv[0], secondpart, str(year), base_link_csv[1]])
        lil_df = pd.read_csv(URL, encoding='ISO-8859-1')
    else:
        URL = ''.join([firstpart, base_link_excel[0], secondpart, str(year), base_link_excel[1]])
        lil_df = pd.read_excel(URL, encoding='ISO-8859-1')
    print('\n')
    lil_df.rename(columns = rename_dict, inplace = True)
    for drop in drops:
        try:
            lil_df.drop([drop], axis = 1, inplace = True)
        except:
            pass
    print(year)
    print('new columns this year:', list(set(lil_df.columns) - set(all_columns)))
    print('missing columns this year:', list(set(all_columns) - set(lil_df.columns)))
    all_columns.extend(list(lil_df.columns))
    all_columns = list(set(all_columns))
    df = pd.concat([df, lil_df], axis = 0, sort = True)
print('\n')
print(len(df))
print('\n')
df.reset_index(drop = True, inplace = True)
df.isnull().sum()[(df.isnull().sum() > 0)].sort_values(ascending = False)



2003
new columns this year: ['ac_evasv', 'ac_cgdir', 'race', 'comppct', 'haircolr', 'addrtyp', 'radio', 'ac_assoc', 'cs_drgtr', 'zip', 'recstat', 'pf_wall', 'pf_pepsp', 'ycoord', 'ac_inves', 'rf_bulg', 'rf_othsw', 'rf_rfcmp', 'ht_inch', 'stinter', 'sb_hdobj', 'othfeatr', 'sector', 'year', 'arstoffn', 'cs_other', 'xcoord', 'sumissue', 'pf_hands', 'ac_time', 'ac_stsnd', 'ht_feet', 'perstop', 'stname', 'offunif', 'offshld', 'ser_num', 'cs_descr', 'pf_other', 'dob', 'othrweap', 'cs_casng', 'state', 'ac_other', 'explnstp', 'repcmd', 'trhsloc', 'offverb', 'weight', 'pistol', 'city', 'sumoffen', 'addrnum', 'contrabn', 'rescode', 'officrid', 'cs_bulge', 'ac_proxm', 'sb_other', 'othpers', 'sex', 'eyecolor', 'riflshot', 'crossst', 'cs_vcrim', 'premname', 'aptnum', 'rf_vcact', 'revcmd', 'cs_lkout', 'pf_drwep', 'sb_outln', 'pf_hcuff', 'frisked', 'rf_knowl', 'rf_furt', 'inout', 'rf_attir', 'crimsusp', 'machgun', 'timestop', 'rf_verbl', 'post', 'knifcuti', 'asltweap', 'sb_admis', 'rf_vcrim', 'cs_o

Stop Frisk Time                                                 5090238
ASK_FOR_CONSENT_FLG                                             5087785
STOP_ID_ANONY                                                   5087785
CONSENT_GIVEN_FLG                                               5087785
SUPERVISING_ACTION_CORRESPONDING_ACTIVITY_LOG_ENTRY_REVIEWED    5078607
                                                                 ...   
addrnum                                                            6407
crimsusp                                                           1001
sumoffen                                                            522
arstoffn                                                            331
stname                                                               84
Length: 99, dtype: int64

In [3]:
# drop columns missing more than half of rows
drops = list((df.isnull().sum()/len(df))[(df.isnull().sum()/len(df) > 0.5)].index)

df.drop(drops, axis = 1, inplace = True)

In [4]:
df.isnull().sum()[(df.isnull().sum() > 0)].sort_values(ascending = False)

revcmd      36824
othfeatr    36635
stinter     36439
repcmd      36382
asltweap    36096
            ...  
addrnum      6407
crimsusp     1001
sumoffen      522
arstoffn      331
stname         84
Length: 63, dtype: int64

### Stop date

In [5]:
df['datestop'].value_counts()[:10]

2112011    3255
2102012    3200
1062012    3069
4302010    3067
1272012    2983
3092012    2932
3022012    2881
1202012    2874
2012012    2847
2032012    2843
Name: datestop, dtype: int64

In [6]:
df['datestop'].apply(lambda x: len(str(x))).value_counts()

7     3552146
8     1006510
10     506489
19      36096
1           3
Name: datestop, dtype: int64

In [7]:
list(df.loc[df['datestop'].apply(lambda x: len(str(x)) == 1), 'datestop'].values)

[' ', ' ', ' ']

In [8]:
df.loc[df['datestop'].apply(lambda x: len(str(x)) == 1), ['datestop', 'pct']]

,datestop,pct
872565,,
872566,,
5065147,,


In [9]:
df = df[df['datestop'] != ' ']

In [10]:
df.loc[df['datestop'].apply(lambda x: '-' in str(x)), 'datestop'].value_counts()[:10]

2006-02-16    2018
2006-04-12    1995
2006-05-12    1983
2006-02-08    1960
2006-06-06    1959
2006-03-08    1951
2006-02-15    1949
2006-01-20    1929
2006-01-19    1898
2006-02-17    1897
Name: datestop, dtype: int64

In [11]:
df['datestop'].apply(lambda x: type(x)).value_counts()

<class 'int'>                                         4554444
<class 'str'>                                          510701
<class 'pandas._libs.tslibs.timestamps.Timestamp'>      36096
Name: datestop, dtype: int64

In [12]:
alreadymask = [True if ((type(x) != int) & (type(x) != str)) else False for x in df['datestop']]
alreadytimestamp = df[alreadymask]
print(len(alreadytimestamp))

notalready = [True if x == False else False for x in alreadymask]

hyphenmask = [True if '-' in str(x) else False for x in df.loc[notalready, 'datestop']]
hyphendate = df[notalready][hyphenmask]
print(len(hyphendate))

cleanmask = [True if '-' not in str(x) else False for x in df.loc[notalready, 'datestop']]
cleandate = df[notalready][cleanmask]
print(len(cleandate))

hyphendate['datestop'] = [dt.strptime(x, '%Y-%m-%d') for x in hyphendate['datestop']]
print('hyphendate done')
cleandate['datestop'] = ['0' + str(x) if len(str(x)) == 7 else str(x) for x in cleandate['datestop']]
print('0s added to cleandate')
cleandate['datestop'] = [dt.strptime(x, '%m%d%Y') for x in cleandate['datestop']]
print('cleandate done')

df = pd.concat([hyphendate, cleandate], axis = 0, sort = True)
df = pd.concat([df, alreadytimestamp], axis = 0, sort = True)
del cleandate, hyphendate, alreadytimestamp

36096
506489
4558656
hyphendate done
0s added to cleandate
cleandate done


In [13]:
df['dayofmonth'] = [x.day for x in df['datestop']]
df['month'] = [x.strftime('%B') for x in df['datestop']]
df['weekday'] = [x.strftime('%A') for x in df['datestop']]
df['ordinalday'] = [x.toordinal() for x in df['datestop']]

In [14]:
df['datestop'].sort_values()[:10]

536581    1900-12-31
1340839   1900-12-31
1160274   1900-12-31
1160187   1900-12-31
392384    1900-12-31
1365909   1900-12-31
1376784   1900-12-31
1126117   1900-12-31
1123584   1900-12-31
1112750   1900-12-31
Name: datestop, dtype: datetime64[ns]

In [16]:
tooearly = dt.strptime('1900-12-31', '%Y-%m-%d')
all_early = df.loc[df['datestop'] == tooearly, 'pct']
print(len(all_early))
all_early.value_counts()

56


113    12
75      4
23      4
28      2
43      2
62      2
34      2
101     2
79      2
81      2
70      2
68      2
73      1
72      1
30      1
83      1
76      1
84      1
88      1
90      1
94      1
100     1
103     1
26      1
17      1
52      1
6       1
120     1
123     1
66      1
Name: pct, dtype: int64

In [17]:
df = df[df['datestop'] != tooearly]

In [18]:
df['datestop'].sort_values()[:10]

0     2003-01-01
340   2003-01-01
339   2003-01-01
338   2003-01-01
337   2003-01-01
336   2003-01-01
335   2003-01-01
341   2003-01-01
334   2003-01-01
332   2003-01-01
Name: datestop, dtype: datetime64[ns]

In [19]:
df['ordinalday'].describe()

count    5.101185e+06
mean     7.334317e+05
std      1.058512e+03
min      7.312160e+05
25%      7.325940e+05
50%      7.334900e+05
75%      7.342610e+05
max      7.374240e+05
Name: ordinalday, dtype: float64

In [20]:
(df['ordinalday'].max() - df['ordinalday'].min())/365.25

16.996577686516083

In [21]:
df.to_csv('data/SQF_2003_through_2019.csv', index = False)